# Домашнее задание № 2. Мешок слов

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

import math
from razdel import tokenize
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_ru = stopwords.words('russian')
from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sara7\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)

In [4]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

##### векторы с дефолтной токенизацией:

In [5]:
default_vect = TfidfVectorizer(min_df=10, max_df=0.3)

# train
x_dv = default_vect.fit_transform(train.comment)
y_dv = train.toxic.values

# test
x_test_dv = default_vect.transform(test.comment)
y_test_dv = test.toxic.values

In [6]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(x_dv, y_dv)
preds = clf.predict(x_test_dv)

C:\Users\sara7\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [7]:
print(classification_report(y_test_dv, preds))

              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87       954
         1.0       0.74      0.77      0.75       488

    accuracy                           0.83      1442
   macro avg       0.81      0.82      0.81      1442
weighted avg       0.83      0.83      0.83      1442



##### кастомная токенизация  razdel.tokenize:

In [8]:
def razdel_tokenizer(set_of_comments):
    tokens = []
    for item in set_of_comments:
        new_tokens = [word.text for word in list(tokenize(item))]
        tokens.append(new_tokens)
    return(tokens)

def razdel_tokenizer1(thing):
    return(thing)

In [9]:
razdel_vect = TfidfVectorizer(lowercase=False,
                                    tokenizer=razdel_tokenizer1,
                                    max_df=0.3,
                                    min_df=10)
# train
train_rt = razdel_tokenizer(train.comment)
x_rt = razdel_vect.fit_transform(train_rt)
y_rt = train.toxic.values

print('x_rt: ', x_rt.shape)
print('y_rt: ', y_rt.shape)

# test
test_rt = razdel_tokenizer(test.comment)
x_test_rt = razdel_vect.transform(test_rt)
y_test_rt = test.toxic.values

print('x_test_rt: ', x_test_rt.shape)
print('y_test_rt: ', y_test_rt.shape)

x_rt:  (12970, 3464)
y_rt:  (12970,)
x_test_rt:  (1442, 3464)
y_test_rt:  (1442,)


In [10]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(x_rt, y_rt)
preds = clf.predict(x_test_rt)
print(classification_report(y_test_rt, preds))

              precision    recall  f1-score   support

         0.0       0.87      0.82      0.84       954
         1.0       0.68      0.75      0.71       488

    accuracy                           0.80      1442
   macro avg       0.77      0.79      0.78      1442
weighted avg       0.80      0.80      0.80      1442



_Каждый раз получается, что дефолтная векторизация немного лучше._

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [11]:
tabledata = [['я и ты',1,1,1,0,0,0],
             ['ты и я',1,1,1,0,0,0],
             ['я, я и только я',3,0,1,1,0,0],
             ['только не я',1,0,0,1,1,0],
             ['он',0,0,0,0,0,1]]

In [12]:
df = pd.DataFrame(tabledata, columns = ['phrase','я','ты','и','только','не','он'])
df

,phrase,я,ты,и,только,не,он
0,я и ты,1,1,1,0,0,0
1,ты и я,1,1,1,0,0,0
2,"я, я и только я",3,0,1,1,0,0
3,только не я,1,0,0,1,1,0
4,он,0,0,0,0,0,1


In [13]:
# сначала преобразуем значения в относительные

def make_relational(row):
    phrase_len = sum([item for item in row if type(item) != str])
    new_values = [item/phrase_len for item in row if type(item) != str]
    new_row = [row[0]]
    new_row.extend(new_values)
    return new_row

In [14]:
relational_tabledata = [make_relational(row) for row in tabledata]

In [15]:
rel_df = pd.DataFrame(relational_tabledata, columns = ['phrase','я','ты','и','только','не','он'])
rel_df

,phrase,я,ты,и,только,не,он
0,я и ты,0.333333,0.333333,0.333333,0.000000,0.000000,0.0
1,ты и я,0.333333,0.333333,0.333333,0.000000,0.000000,0.0
2,"я, я и только я",0.600000,0.000000,0.200000,0.200000,0.000000,0.0
3,только не я,0.333333,0.000000,0.000000,0.333333,0.333333,0.0
4,он,0.000000,0.000000,0.000000,0.000000,0.000000,1.0


In [16]:
# cчитаем значение df

def df_count(word):
    df = sum([1 for v in list(rel_df[word].values) if v > 0])
    return(df)

In [17]:
def make_tf_idf(data):
    N = len(data) # общее кол-во документов в выборке (кол-во строк в rel_df)

    words = ['я', 'ты', 'и', 'только', 'не', 'он']

    tf_idf_tabledata = []

    for row in data:
        index = data.index(row)
        tf_idf_row = [row[0]]

        for word in words:
            tf = row[words.index(word)+1] # частотность слова x в документе y (значения в rel_df)
            df = df_count(word) # кол-во документов, где встречается x

            tf_idf = tf * math.log(N/df) # Ф О Р М У Л А !

            tf_idf_row.append(tf_idf)

        tf_idf_tabledata.append(tf_idf_row)
        
    return tf_idf_tabledata

In [18]:
tf_idf_tabledata = make_tf_idf(relational_tabledata)

In [19]:
tf_idf_df = pd.DataFrame(tf_idf_tabledata, columns = ['phrase','я','ты','и','только','не','он'])

tf_idf_df

,phrase,я,ты,и,только,не,он
0,я и ты,0.074381,0.30543,0.170275,0.000000,0.000000,0.000000
1,ты и я,0.074381,0.30543,0.170275,0.000000,0.000000,0.000000
2,"я, я и только я",0.133886,0.00000,0.102165,0.183258,0.000000,0.000000
3,только не я,0.074381,0.00000,0.000000,0.305430,0.536479,0.000000
4,он,0.000000,0.00000,0.000000,0.000000,0.000000,1.609438


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [20]:
# 1
# Tfidf Vectorizer

Tfidf_Vect = TfidfVectorizer(decode_error='ignore',
                             stop_words=stopwords_ru,
                             min_df=10,
                             ngram_range=(1,2),
                             max_df=0.3)

# train
x_tfidf = Tfidf_Vect.fit_transform(train.comment)
y_tfidf = train.toxic.values

# test
x_test_tfidf = Tfidf_Vect.transform(test.comment)
y_test_tfidf = test.toxic.values

In [21]:
# DecisionTree Classifier

dec_tree_clf = DecisionTreeClassifier(splitter='random', class_weight='balanced')
dec_tree_clf.fit(x_tfidf, y_tfidf)
preds = dec_tree_clf.predict(x_test_tfidf)

print(classification_report(y_test_tfidf, preds))

              precision    recall  f1-score   support

         0.0       0.82      0.79      0.81       954
         1.0       0.62      0.67      0.64       488

    accuracy                           0.75      1442
   macro avg       0.72      0.73      0.72      1442
weighted avg       0.75      0.75      0.75      1442



In [22]:
dec_tree_probs = dec_tree_clf.predict_proba(x_test_tfidf)
dec_tree_count_list = Counter()

for i in range(len(test.comment)):
    dec_tree_count_list[test.comment[i]] = float(dec_tree_probs[i][1])

dec_tree_count_list.most_common(10)   

[('Говори адрес, петушара, будешь у меня на камеру говорить, что Кузьма - не свинья и на коленях прощения просить\n',
  1.0),
 ('потому что на лурке пишут что он пидор\n', 1.0),
 ('На сигареты и минималка и максималка. Но законы то у нас дырявые и зачастую противоречивые, вот так и суд расценивает попытку продавать сигареты ниже максимальной их стоимости как попытку стимулирования продажи табачных изделий. Так что по факту сигареты продаются всегда по максимальной стоимости. Но это речь о сигаретах, а что с табаком? Я такой инфы не нашёл.\n',
  1.0),
 ('Всем здесь занимали. Мне профсоюз инвалидов умственного труда номерок в эту очередь выдавал!\n',
  1.0),
 ('Мне кажется, что не чувствуется натуральности. Что это именно вещь которую носят, а не одевают ради дубля. Все костюмы и элементы слишком полированные, слишком чистые и идеальные, складывается впечатление, что каждый кадр они надевают новый костюм, а старый тут же в чистку.\n',
  1.0),
 ('А нахуя мы тогда налоги платим, пусть сами

_Довольно подозрительно, что (почти) у всех 100% вер-ть токсичности.._

_Кроме того, не со всеми оценками классификатора я согласна._

_На всякий случай надо присмотреться к результатам:_

In [23]:
dec_tree_probs

array([[1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       ...,
       [0.21931054, 0.78068946],
       [1.        , 0.        ],
       [1.        , 0.        ]])

_Почему-то на поверхности оказываются одинаковые повторяющиеся результаты..._

In [24]:
# посмотрим на результаты из середины

dec_tree_count_list.most_common()[500:550]

[('Вышел покурить на балкон, а тут Ын курит. Постояли, покурили.\n',
  0.7806894575400859),
 ('А в чем тут оскорбление?', 0.7806894575400859),
 ('Я так заплатил 2000 за обезболивание, но зуб так и не восстановил-стало плохо с сердцем(\n',
  0.7806894575400859),
 ('Не давал бы пушистому лизать кофе. Тельце маленькое, может инфаркт ебнуть.\n',
  0.7806894575400859),
 ('Н Е К У К О Л Д Ы И Р А З У\n', 0.7806894575400859),
 ('Алсо, еще один твит про маразматика\n', 0.7806894575400859),
 ('1. Жидовка подтачивает государство. Никогда такого не было, ага. 2. Нахуй фемкам косметика для патриархалок , когда им нужно что-то типа кондиционеров для волос на теле? 3. В чем хамство лаша? На месте этой конторы, глядючи на эту харю, я бы даже и отвечать не стал. 4. Сделала много для фемок, открытую инсту не догадалась зарегать - ЭТО ШИН!!! И кстати почему ее феминоподсписочницы не зафоловили ее в этой инсте, сучки токсичные?\n',
  0.6655358519660756),
 ('Приписываю в гандоны и текущую власть!', 0.6655

_Обнаруживаем, что в середине всё же есть другие, не абсолютные значения (которые, кстати, тоже повторяются)._

_Уже сейчас, безотносительно к результатам второго классификатора, можно сказать, что, несмотря на хорошие показатели метрик, этот классификатор работает странно, если не сказать некорректно..._

In [25]:
# 2
# Count Vectorizer

Count_Vect = CountVectorizer(decode_error='ignore',
                                 stop_words=stopwords_ru,
                                 min_df=10,
                                 ngram_range=(1,2),
                                 max_df=0.3)

# train
x_count = Count_Vect.fit_transform(train.comment)
y_count = train.toxic.values

# test
x_test_count = Count_Vect.transform(test.comment)
y_test_count = test.toxic.values

In [26]:
# KNeighbors Classifier

kn_clf = KNeighborsClassifier(n_neighbors=8, metric='cosine')
kn_clf.fit(x_count, y_count)
preds = kn_clf.predict(x_test_count)

print(classification_report(y_test_count, preds))

              precision    recall  f1-score   support

         0.0       0.76      0.84      0.80       954
         1.0       0.60      0.48      0.53       488

    accuracy                           0.72      1442
   macro avg       0.68      0.66      0.67      1442
weighted avg       0.71      0.72      0.71      1442



_По метрикам DecisionTree как будто немного лучше, сейчас посмотрим на оценку для конкретных текстов_

In [27]:
kn_probs = kn_clf.predict_proba(x_test_count)
kn_count_list = Counter()

for i in range(len(test.comment)):
    kn_count_list[test.comment[i]] = float(kn_probs[i][1])

kn_count_list.most_common(10)   

[('потому что на лурке пишут что он пидор\n', 1.0),
 ('где расчехляет кулаки, ты врешь 4пик Заебала пиздеть, русня', 1.0),
 ('Бля, заебал он уже, во всякой бочке затычка. Где его нестиранные штаны в дальнейшем увидим?\n',
  1.0),
 ('мы покурим шышки Оп -- малолетний дебил. Конопля пригодная для хозяства и конопля для курения -- разные. Хозяйственую нельзя курить каннабиноидов в ней нет.\n',
  1.0),
 ('Да ладно тебе. Чуть что сразу модератор\n', 1.0),
 ('У него к-к-к-комбо: нигер, педераст и баба\n', 1.0),
 ('Союз дети скрепляют, но твоя шлюха в них не может.\n', 1.0),
 ('АХАХАХАХАХАХАХ Ты в очередной раз доказываешь насколько ты тупой долбоёб.\n',
  1.0),
 ('Такой себе Антихрист для русских\n', 1.0),
 ('Что блять за конференция мамашек?\n', 1.0)]

In [28]:
kn_probs

array([[0.25 , 0.75 ],
       [1.   , 0.   ],
       [0.875, 0.125],
       ...,
       [0.375, 0.625],
       [1.   , 0.   ],
       [1.   , 0.   ]])

_Этот классификатор, хоть и проигрывает по значениям метрик, показывает более адекватные результаты: я в большей степени согласна с его выборкой токсичных комментов, нежели с результатами предыдущего классфикатора._

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression и Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [51]:
new_stopwords = ['очень'] # потому что позже оно вылезает в частотных и лучше его сразу убрать
new_stopwords.extend(stopwords_ru)

In [42]:
vect = TfidfVectorizer(ngram_range=(1, 3),
                             min_df=5,
                             max_df=0.2,
                             stop_words=new_stopwords)

# train
x_train = vect.fit_transform(train.comment)
y_train = train.toxic.values

# test
x_test = vect.transform(test.comment)
y_test = test.toxic.values

In [43]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [44]:
wordlist = [morph.parse(word)[0].normal_form for word in vect.get_feature_names()]
len(wordlist)

7934

In [45]:
log_reg_clf = LogisticRegression(C=0.1, class_weight='balanced')
log_reg_clf.fit(x_train, y_train)
preds = log_reg_clf.predict(x_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87       954
         1.0       0.74      0.78      0.76       488

    accuracy                           0.83      1442
   macro avg       0.81      0.82      0.82      1442
weighted avg       0.84      0.83      0.84      1442



C:\Users\sara7\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [46]:
# coefficient of the features in the decision function.

log_reg_coef = log_reg_clf.coef_
log_reg_coef_counter_list = Counter()
for i in range(len(wordlist)):
    log_reg_coef_counter_list[wordlist[i]] = log_reg_coef[0][i]

In [47]:
log_reg_coef_counter_list.most_common(5) 

[('хохлов', 1.546664875990788),
 ('блять', 1.102446652268356),
 ('блядь', 1.0808134849701856),
 ('пиздец', 1.0174797919903233),
 ('сук', 1.0032950220542616)]

In [48]:
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=20, )
rf_clf.fit(x_train, y_train)
preds = rf_clf.predict(x_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.68      1.00      0.81       954
         1.0       0.97      0.06      0.12       488

    accuracy                           0.68      1442
   macro avg       0.82      0.53      0.46      1442
weighted avg       0.77      0.68      0.57      1442



In [49]:
# The impurity-based feature importances.

rf_coef = rf_clf.feature_importances_
rf_coef_counter_list = Counter()
for i in range(len(wordlist)):
    rf_coef_counter_list[wordlist[i]] = rf_coef[i]

In [50]:
rf_coef_counter_list.most_common(5)

[('блядь', 0.021707386556318493),
 ('пиздец', 0.020068699935043517),
 ('хохлов', 0.017702619745936853),
 ('блять', 0.01677294246402691),
 ('сук', 0.01657295451677767)]